<a href="https://colab.research.google.com/github/soham7707/DeepSectorAI/blob/main/test01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Full Reconfigured Code for BullishPeriodAnalysis.ipynb with Bi-LSTM

# --- 1. Install Dependencies ---
!pip install arch -q
!pip install yfinance -q
!pip install tensorflow -q

# --- 2. Import Libraries ---
import pandas as pd
import numpy as np
import yfinance as yf
from arch import arch_model
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
from IPython.display import display

# --- 3. Define Sectors and Fetch Data ---
sectors = {
    'IT': ['TCS.NS', 'INFY.NS', 'HCLTECH.NS', 'WIPRO.NS'],
    'Banking': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
    'Pharma': ['SUNPHARMA.NS', 'CIPLA.NS', 'DRREDDY.NS'],
    'Auto': ['TATAMOTORS.NS', 'M&M.NS', 'MARUTI.NS'],
    'FMCG': ['HINDUNILVR.NS', 'ITC.NS', 'DABUR.NS']
}

start_date = "2020-04-01"
end_date = "2021-10-01"

def fetch_sector_data(sectors):
    all_data = {}
    for sector, stocks in sectors.items():
        print(f"Fetching data for {sector} sector...")
        data = yf.download(stocks, start=start_date, end=end_date, progress=False)['Close']
        data = data.ffill().bfill() # Forward and backward fill
        all_data[sector] = data
    return all_data

sector_data = fetch_sector_data(sectors)

# --- 4. Calculate Log Returns ---
returns_data = {}
for sector, df in sector_data.items():
    returns = np.log(df / df.shift(1)).dropna()
    returns_data[sector] = returns

# --- 5. GARCH Model Implementation ---
def estimate_garch_volatility(returns_series):
    # Scaling by 100 helps optimizer convergence
    model = arch_model(returns_series * 100, vol='GARCH', p=1, q=1, rescale=False)
    res = model.fit(disp='off')
    return res.conditional_volatility / 100

garch_volatility_data = {}
for sector, returns_df in returns_data.items():
    print(f"Estimating GARCH volatility for {sector} sector...")
    sector_vols = pd.DataFrame(index=returns_df.index)
    for stock in returns_df.columns:
        try:
            vol = estimate_garch_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error processing GARCH for {stock}: {e}")
    garch_volatility_data[sector] = sector_vols

# --- 6. Bidirectional LSTM Model Implementation ---
def prepare_lstm_data(series, lookback=20):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(series.values.reshape(-1, 1))
    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i-lookback:i])
        y.append(scaled_data[i])
    X = np.array(X)
    y = np.array(y)
    return X, y, scaler

def build_bilstm_model(input_shape):
    model = Sequential([
        Bidirectional(LSTM(units=50, return_sequences=False), input_shape=input_shape),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

def predict_bilstm_volatility(returns_series, lookback=20):
    X, y, scaler = prepare_lstm_data(returns_series, lookback)
    model = build_bilstm_model((X.shape[1], X.shape[2]))
    model.fit(X, y, epochs=10, batch_size=16, verbose=0)
    y_pred_scaled = model.predict(X, verbose=0)
    y_pred = scaler.inverse_transform(y_pred_scaled).flatten()
    predicted_volatility = pd.Series(y_pred).rolling(window=lookback).std()
    predicted_volatility.index = returns_series.index[lookback:]
    return predicted_volatility

bilstm_volatility_data = {}
for sector, returns_df in returns_data.items():
    print(f"Predicting Bi-LSTM volatility for {sector} sector...")
    sector_vols = pd.DataFrame()
    for stock in returns_df.columns:
        try:
            vol = predict_bilstm_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error with {stock}: {e}")
    bilstm_volatility_data[sector] = sector_vols

# --- 7. Beta Calculation ---
nifty = yf.download("^NSEI", start=start_date, end=end_date, progress=False)['Close']
nifty_returns = np.log(nifty / nifty.shift(1)).dropna()
nifty_returns_df = pd.DataFrame(nifty_returns)
nifty_returns_df.columns = ['Market']

def compute_beta(stock_returns, market_returns_df):
    aligned = pd.concat([stock_returns, market_returns_df], axis=1).dropna()
    aligned.columns = ['Stock', 'Market']
    X = sm.add_constant(aligned['Market'])
    y = aligned['Stock']
    model = sm.OLS(y, X).fit()
    return model.params['Market']

beta_values = {}
for sector, returns_df in returns_data.items():
    print(f"Computing Beta for {sector} sector...")
    sector_betas = {}
    for stock in returns_df.columns:
        try:
            beta = compute_beta(returns_df[stock], nifty_returns_df)
            sector_betas[stock] = beta
        except Exception as e:
            print(f"Error with {stock}: {e}")
    beta_values[sector] = sector_betas

# --- 8. Sharpe Ratio Calculation ---
sharpe_ratios = {}
for sector, df in returns_data.items():
    sector_sharpes = {}
    for stock in df.columns:
        r = df[stock].dropna()
        mean_return = r.mean()
        std_return = r.std()
        sharpe = (mean_return / std_return if std_return != 0 else 0) * np.sqrt(252) # Annualized
        sector_sharpes[stock] = sharpe
    sharpe_ratios[sector] = sector_sharpes

# --- 9. Feature Compilation and DataFrame Creation ---
features = []
for sector in returns_data:
    for stock in returns_data[sector].columns:
        try:
            beta = beta_values[sector][stock]
            garch_mean = garch_volatility_data[sector][stock].mean()
            bilstm_mean = bilstm_volatility_data[sector][stock].mean()
            sharpe = sharpe_ratios[sector][stock]
            features.append([sector, stock, beta, garch_mean, bilstm_mean, sharpe])
        except KeyError:
            print(f"Skipping {stock} in sector {sector} due to missing data.")

features_df = pd.DataFrame(features, columns=['Sector', 'Stock', 'Beta', 'GARCH_Vol', 'BiLSTM_Vol', 'Sharpe'])
features_df.dropna(inplace=True) # Ensure no NaN values before modeling

# --- 10. Risk Score Modeling ---
# Model 1: All features
X = features_df[['Beta', 'GARCH_Vol', 'BiLSTM_Vol']]
y = features_df['Sharpe']
model_all = LinearRegression().fit(X, y)
features_df['Risk_Score'] = X @ model_all.coef_

# Model 2: GARCH and Bi-LSTM
X_gb = features_df[['GARCH_Vol', 'BiLSTM_Vol']]
model_gb = LinearRegression().fit(X_gb, y)
features_df['Risk_Score_BiLSTM_GARCH'] = X_gb @ model_gb.coef_

# Model 3: Beta and Bi-LSTM
X_lb = features_df[['Beta', 'BiLSTM_Vol']]
model_lb = LinearRegression().fit(X_lb, y)
features_df['Risk_Score_BiLSTM_Beta'] = X_lb @ model_lb.coef_

# Model 4: GARCH and Beta
X_b = features_df[['Beta', 'GARCH_Vol']]
model_b = LinearRegression().fit(X_b, y)
features_df['Risk_Score_GARCH_Beta'] = X_b @ model_b.coef_

print("\n--- Feature DataFrame with Risk Scores ---")
display(features_df.head())

# --- 11. Analysis and Ranking Functions ---
def compare_sector_models(sector_name, top_n=5):
    df = features_df[features_df['Sector'] == sector_name].copy()
    models = ['Risk_Score', 'Risk_Score_BiLSTM_GARCH', 'Risk_Score_BiLSTM_Beta', 'Risk_Score_GARCH_Beta']
    print(f"\n📊 Sector: {sector_name}")
    for model in models:
        # A lower risk score is considered "safer"
        print(f"🔹 Top {top_n} safest stocks using {model.replace('_', ' ')}:")
        display(df.sort_values(model).head(top_n)[['Stock', model]])

def sector_ranking_table(sector_name):
    df = features_df[features_df['Sector'] == sector_name].copy()
    ranking_df = df[['Stock']].copy()
    for col in ['Risk_Score', 'Risk_Score_BiLSTM_GARCH', 'Risk_Score_BiLSTM_Beta', 'Risk_Score_GARCH_Beta']:
        ranking_df[col + '_Rank'] = df[col].rank(method='min')
    return ranking_df.sort_values('Risk_Score_Rank')

# --- 12. Display Results ---
for sector in sectors:
    compare_sector_models(sector, top_n=3)

print("\n\n--- Example Ranking Table for Banking Sector ---")
display(sector_ranking_table("Banking"))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 12.0 MB/s eta 0:00:00
Fetching data for IT sector...


/tmp/ipython-input-2706212241.py:36: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date, progress=False)['Close']


Fetching data for Banking sector...


/tmp/ipython-input-2706212241.py:36: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date, progress=False)['Close']


Fetching data for Pharma sector...


...

,Stock,Risk_Score_BiLSTM_Beta
16,ITC.NS,-0.171232
14,DABUR.NS,-0.126679
15,HINDUNILVR.NS,-0.123998


🔹 Top 3 safest stocks using Risk Score GARCH Beta:


,Stock,Risk_Score_GARCH_Beta
15,HINDUNILVR.NS,0.866272
14,DABUR.NS,0.873732
16,ITC.NS,1.065199




--- Example Ranking Table for Banking Sector ---


,Stock,Risk_Score_Rank,Risk_Score_BiLSTM_GARCH_Rank,Risk_Score_BiLSTM_Beta_Rank,Risk_Score_GARCH_Beta_Rank
5,HDFCBANK.NS,1.0,1.0,4.0,1.0
6,ICICIBANK.NS,2.0,3.0,2.0,2.0
7,KOTAKBANK.NS,3.0,2.0,3.0,3.0
4,AXISBANK.NS,4.0,4.0,1.0,4.0
